In [1]:
import pandas as pd
import requests
import numpy as np
from math import pi, sin, cos
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)
import time


In [2]:
units_list_csv = "C:\\Users\\John\\Documents\\notGitHub\\Address_Residential_Units.csv"
units = pd.read_csv(units_list_csv)
print(units.shape)
units.head()

(236544, 9)


,OBJECTID,UNIT_ID,ADDRESS_ID,FULLADDRESS,SSL,STATUS,UNITNUM,UNITTYPE,METADATA_ID
0,1001,736,145729,3426 MINNESOTA AVENUE SE,5433 2002,ACTIVE,2,CONDO,316134
1,1002,738,13061,5117 A STREET SE,5308 2001,ACTIVE,101,CONDO,316136
2,1003,739,74704,11 46TH STREET SE,5346 2011,ACTIVE,B1,CONDO,500789
3,1004,740,58146,17 46TH STREET SE,5346 2021,ACTIVE,1,CONDO,316138
4,1005,741,43800,29 46TH STREET SE,5346 2031,ACTIVE,1,CONDO,316139


In [3]:
address_list_csv = "C:\\Users\\John\\Documents\\notGitHub\\Address_Points.csv"
addresses = pd.read_csv(address_list_csv)
print(addresses.shape)
addresses.head()

add_X_Y = addresses[['X','Y','ADDRESS_ID']]
add_X_Y.head()

c:\programdata\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(147114, 52)


,X,Y,ADDRESS_ID
0,-76.959324,38.894003,36798
1,-76.959042,38.894601,36801
2,-76.959861,38.890442,36847
3,-76.956351,38.894391,36942
4,-76.956809,38.894305,37449


In [4]:
combo = units.merge(add_X_Y, how='left',on='ADDRESS_ID')

In [5]:
combo[['X','Y']].dropna(axis=0, how='any').describe()

# everything matches!  DC government, I love you.

,X,Y
count,236544.000000,236544.000000
mean,-77.019537,38.907405
std,0.035755,0.030278
min,-77.111146,38.819327
25%,-77.042762,38.894627
50%,-77.024201,38.908839
75%,-76.998204,38.926736
max,-76.911176,38.987324


In [6]:
df = combo[['X','Y','UNITNUM','ADDRESS_ID']].groupby('ADDRESS_ID').agg({
    'X':'mean',
    'Y':'mean',
    'UNITNUM':'count'
}).copy()
print(df.shape)
df.head()

(21199, 3)


,X,Y,UNITNUM
ADDRESS_ID,,,
7,-76.995947,38.905030,5
21,-76.995993,38.891860,2
24,-76.995926,38.887205,1
28,-76.994867,38.937997,4
78,-76.995913,38.897539,1


In [7]:
df.describe()

,X,Y,UNITNUM
count,21199.000000,21199.000000,21199.000000
mean,-77.005909,38.901620,11.158262
std,0.034504,0.031387,35.493422
min,-77.111146,38.819327,1.000000
25%,-77.030590,38.885044,2.000000
50%,-77.006160,38.907066,4.000000
75%,-76.984540,38.921897,7.000000
max,-76.911176,38.987324,1181.000000


In [8]:
df.sort_values('UNITNUM',ascending=False, inplace=True)

In [9]:
key_path = "C:\\Users\\John\\Documents\\notGitHub\\API_key.txt"
key_file = open(key_path, 'r')
api_key = key_file.readline()

In [10]:

string1 = "https://mobilityscore.transitscreen.io/api"
string2 = "/v1/locations.json?coordinates="
string3 = "&key="
string4 = "&geojson=true&debug=true"

def api_string(coords):
    new_string = string1+string2+coords+string3+api_key+string4
    return new_string

df['loc_str'] = df.apply(lambda x: (str(x.Y)+","+str(x.X)), axis=1)
df['request'] = df.loc_str.apply(api_string)
df.head()

,X,Y,UNITNUM,loc_str,request
ADDRESS_ID,,,,,
243225,-77.037265,38.936436,1181,"38.93643575527931,-77.03726510819578",https://mobilityscore.transitscreen.io/api/v1/...
284325,-77.024650,38.920318,917,"38.92031826898436,-77.02464966400156",https://mobilityscore.transitscreen.io/api/v1/...
294461,-77.081959,38.935867,671,"38.93586687687617,-77.08195893397365",https://mobilityscore.transitscreen.io/api/v1/...
235844,-77.035880,38.923626,664,"38.92362582942218,-77.03588045315645",https://mobilityscore.transitscreen.io/api/v1/...
220630,-77.061250,38.943732,638,"38.94373173501572,-77.06125006503876",https://mobilityscore.transitscreen.io/api/v1/...


In [11]:
# base = requests.get(df.loc[294461,'request'])

df['Info'] = None
df.head(5)

,X,Y,UNITNUM,loc_str,request,Info
ADDRESS_ID,,,,,,
243225,-77.037265,38.936436,1181,"38.93643575527931,-77.03726510819578",https://mobilityscore.transitscreen.io/api/v1/...,None
284325,-77.024650,38.920318,917,"38.92031826898436,-77.02464966400156",https://mobilityscore.transitscreen.io/api/v1/...,None
294461,-77.081959,38.935867,671,"38.93586687687617,-77.08195893397365",https://mobilityscore.transitscreen.io/api/v1/...,None
235844,-77.035880,38.923626,664,"38.92362582942218,-77.03588045315645",https://mobilityscore.transitscreen.io/api/v1/...,None
220630,-77.061250,38.943732,638,"38.94373173501572,-77.06125006503876",https://mobilityscore.transitscreen.io/api/v1/...,None


In [14]:
from math import floor

def pull_to_point(n, m):
    """Pull until total number of pulls is n.  Save after m pulls."""
    resume_point = df[df.Info.notnull()].shape[0]
    
    t = tqdm_notebook(range(floor(n/m)), initial=floor(resume_point/m), position=0)
    
    for iterator in t:
        
        new_index = df[df['Info'].isnull()].sample(m).index
        for i in tqdm_notebook(new_index, leave=False, position=1):
            df.at[i, 'Info'] = requests.get(df.loc[i,'request'])
        df.to_pickle('checkpoint.pickle')

In [16]:
df = pd.read_pickle('checkpoint.pickle')
pull_to_point(20000, 100)

In [17]:
df.shape[0]

21199

In [37]:
# for some reason using the process above broke down when the sample was 
# the entire rest of the contents of df[df.Info.isnull()]
# so I use this instead for the last bit

df_sample = df[df['Info'].isnull()].copy()
df_sample.head()

for i in tqdm_notebook(df_sample.index):
    df.at[i, 'Info'] = requests.get(df_sample.loc[i,'request'])

df_sample.head()

df.Info.isnull().describe()

count     21199
unique        1
top       False
freq      21199
Name: Info, dtype: object

In [199]:
df.to_pickle('points_pickle')